# Анализ текстовых комментариев

# Описание задачи

Необходимо по имеющимся данным построить модель, которая будет классифицировать текстовые комментарии на негативные и позитивные. Модель нужно построить со значением метрики качества $F1$ не меньше 0,75.

# План работы

1. Подготовка
2. Обучение
    1. Обучение без учёта дисбаланса классов
        1. Логистическая регрессия
        2. Решающее дерево
        3. Случайный лес
    2. Обучение с учётом дисбаланса классов
        1. Логистическая регрессия
        2. Решающее дерево
        3. Случайный лес
3. Выводы

# Описание данных

**Признаки:**

* text - текст комментария
* toxic - целевой признак

## Подготовка

In [23]:
#Импорт библиотек и функций
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 

from sklearn.metrics import f1_score 
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

import spacy
import re 

In [24]:
#Загрузка данных
data = pd.read_csv('datasets/toxic_comments.csv')

In [25]:
#Просмотр данных
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [26]:
#Проверка данных на наличие пропусков
data.isna().sum()

text     0
toxic    0
dtype: int64

In [ ]:
#Создание объекта nlp
nlp = spacy.load("en_core_web_sm")

In [28]:
#Создание корпуса текстов
corpus = list(data['text'])

In [29]:
#Функция лемматизации текстов
def lemmatize(text):   
    doc = nlp(text)
    lemm_list = [token.lemma_ for token in doc]
    lemm_text = " ".join(lemm_list)        
    return lemm_text

In [30]:
#Функция очистки текстов от посторонних символов
def clear_text(text): 
    clr_text = " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split())
    return lemmatize(clr_text)

In [31]:
#Последовательная передача текстов функции лемматизации
data['lemm_text'] = data['text'].progress_apply(lambda x: clear_text(x))  

100%|██████████| 159571/159571 [36:32<00:00, 72.79it/s] 


In [32]:
#Повторный просмотр данных
data

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man I m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,and for the second time of ask when your view ...
159567,You should be ashamed of yourself \n\nThat is ...,0,you should be ashamed of yourself that be a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm there s no actual article for pros...
159569,And it looks like it was actually you who put ...,0,and it look like it be actually you who put on...


В датасете появился столбец "lemm_text"

In [109]:
#Загрузка списка стоп-слов
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VAP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
#Вызов функции stopwords.words c аргументом 'english'
stop_words = set(nltk_stopwords.words('english')) 

In [36]:
#Разбиение данных на обучающую и тестовую части и получение признаков обучающей выборки и целевого признака. 
data_train, data_test = train_test_split(data, test_size=0.25, random_state=12345) 
corpus_train = data_train['lemm_text']
count_tf_idf_train = TfidfVectorizer(stop_words=stop_words)
tf_idf_train = count_tf_idf_train.fit_transform(corpus_train) 
features_train = tf_idf_train
target_train = data_train['toxic']   

In [37]:
#Получение признаков тестовой выборки и целевого признака
corpus_test = data_test['lemm_text']
tf_idf_test = count_tf_idf_train.transform(corpus_test)
features_test = tf_idf_test
target_test = data_test['toxic']

## Обучение

### Обучение без учёта дисбаланса классов

#### Логистическая регрессия

In [38]:
#Создание объекта модели логистической регрессии
model_lr = LogisticRegression(random_state=12345)

In [39]:
#Обучение модели логистической регрессии
model_lr.fit(features_train, target_train)

LogisticRegression(random_state=12345)

In [40]:
#Предсказания модели логистической регрессии
predictions = model_lr.predict(features_test)

In [41]:
#Метрика F1 модели логистической регрессии
print(f1_score(target_test, predictions))

0.7466433158201986


#### Решающее дерево

In [42]:
#Создание объекта модели решающего дерева
model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=200)

In [43]:
#Обучение модели решающего дерева
model_dtc.fit(features_train, target_train)

DecisionTreeClassifier(max_depth=200, random_state=12345)

In [44]:
#предсказания модели решающего дерева
predictions = model_dtc.predict(features_test)

In [45]:
#Метрика F1 модели решающего дерева
print(f1_score(target_test, predictions))

0.7263144363028666


#### Случайный лес

In [66]:
#Создание объекта модели случайного леса
model_rfc = RandomForestClassifier(random_state=12345, n_estimators=8, max_depth=200)

In [67]:
#Обучение модели случайного леса
model_rfc.fit(features_train, target_train)

RandomForestClassifier(max_depth=200, n_estimators=8, random_state=12345)

In [68]:
#Предсказания модели случайного леса
predictions = model_rfc.predict(features_test)

In [69]:
#Метрика F1 модели случайного леса
print(f1_score(target_test, predictions))

0.5704165963906224


### Обучение с учётом дисбаланса классов

#### Логистическая регрессия

In [70]:
#Доли классов
target_train.value_counts(normalize=True)

0    0.898578
1    0.101422
Name: toxic, dtype: float64

In [71]:
#Функция увеличения выборки для борьбы с дисбалансом классов

def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [72]:
#Обращение к функции увеличения выборки для устранения дисбаланса
features_upsampled_train, target_upsampled_train = upsample(data_train['lemm_text'], target_train, 5)

In [73]:
#Повторный просмотр долей классов
target_upsampled_train.value_counts(normalize=True)

0    0.639244
1    0.360756
Name: toxic, dtype: float64

In [74]:
#Создание корпуса текстов обучающей выборки
corpus_train = features_upsampled_train.values.astype('U')

In [75]:
#Создание счетчика с указание стоп-слов
count_tf_idf_train = TfidfVectorizer(stop_words=stop_words)

In [76]:
#Подсчет TF-IDF для корпуса текстов обучающей выборки
tf_idf_train = count_tf_idf_train.fit_transform(corpus_train) 

In [77]:
#Новый набор признаков обучающей выборки после устранения дисбаланса классов
features_upsampled_train_tf_idf = tf_idf_train

In [78]:
#Создание объекта модели логистической регрессии
model_lr = LogisticRegression(random_state=12345)

In [79]:
#Обучение модели логистической регрессии
model_lr.fit(features_upsampled_train_tf_idf, target_upsampled_train)

LogisticRegression(random_state=12345)

In [80]:
#Предсказания модели логистической регрессии
predictions = model_lr.predict(features_test)

In [81]:
#Метрика F1 модели логистической регрессии
print(f1_score(target_test, predictions))

0.7777271692097472


#### Решающее дерево

In [82]:
#Доли классов
target_train.value_counts(normalize=True)

0    0.898578
1    0.101422
Name: toxic, dtype: float64

In [83]:
#Обращение к функции увеличения выборки для устранения дисбаланса
features_upsampled_train, target_upsampled_train = upsample(data_train['lemm_text'], target_train, 10)

In [84]:
#Повторный просмотр доли классов после устранения дисбаланса
target_upsampled_train.value_counts(normalize=True)

1    0.530229
0    0.469771
Name: toxic, dtype: float64

In [85]:
#Создание корпуса текстов обучающей выборки
corpus_train = features_upsampled_train.values.astype('U')

In [86]:
#Создание счетчика с указание стоп-слов
count_tf_idf_train = TfidfVectorizer(stop_words=stop_words)

In [87]:
#Подсчет TF-IDF для корпуса текстов обучающей выборки
tf_idf_train = count_tf_idf_train.fit_transform(corpus_train)

In [88]:
#Новый набор признаков обучающей выборки после устранения дисбаланса классов
features_upsampled_train_tf_idf = tf_idf_train

In [89]:
#Создание объекта модели решающего дерева
model_dtc = DecisionTreeClassifier(random_state=12345,  max_depth=100)

In [90]:
#Обучение объекта модели решающего дерева
model_dtc.fit(features_upsampled_train_tf_idf, target_upsampled_train)

DecisionTreeClassifier(max_depth=100, random_state=12345)

In [91]:
#Предсказания модели решающего дерева
predictions = model_dtc.predict(features_test)

In [92]:
#Метрика F1 модели решающего дерева
print(f1_score(target_test, predictions))

0.6328600405679513


#### Случайный лес

In [93]:
#Доли классов
target_train.value_counts(normalize=True)

0    0.898578
1    0.101422
Name: toxic, dtype: float64

In [94]:
#Обращение к функции увеличения выборки для устранения дисбаланса
features_upsampled_train, target_upsampled_train = upsample(data_train['lemm_text'], target_train, 4)

In [95]:
#Повторный просмотр доли классов после устранения дисбаланса
target_upsampled_train.value_counts(normalize=True)

0    0.688953
1    0.311047
Name: toxic, dtype: float64

In [96]:
#Создание корпуса текстов обучающей выборки
corpus_train = features_upsampled_train.values.astype('U')

In [97]:
##Создание счетчика с указание стоп-слов
count_tf_idf_train = TfidfVectorizer(stop_words=stop_words)

In [98]:
#Подсчет TF-IDF для корпуса текстов обучающей выборки
tf_idf_train = count_tf_idf_train.fit_transform(corpus_train) 

In [99]:
#Новый набор признаков обучающей выборки после устранения дисбаланса классов
features_upsampled_train_tf_idf = tf_idf_train

In [100]:
#Создание объекта модели случайного леса
model_rfc = RandomForestClassifier(random_state=12345, n_estimators=8, max_depth=200)

In [101]:
#Обучение модели случайного леса
model_rfc.fit(features_upsampled_train_tf_idf, target_upsampled_train)

RandomForestClassifier(max_depth=200, n_estimators=8, random_state=12345)

In [102]:
#Предсказания модели случайного леса
predictions = model_rfc.predict(features_test)

In [103]:
#Метрика F1 модели случайного леса.
print(f1_score(target_test, predictions))

0.6398061193577703


## Выводы

Были исследованы следующие модели: модель логистической регрессии (LogisticRegression), модель решающего дерева (DecisionTreeClassifier) и модель случайного леса (RandomForestClassifier). Было необходимо исследовать их с помощью метрики F1. В данной задаче классификации присутствовал дисбаланс классов. Устранением дисбаланса классов и подбором гиперпараметров удалось достичь требуемого значения метрики у модели логистической регрессии. В данной задаче модель логистической регрессии оказалась лучше остальных моделей.